In [82]:
#!pip install pyspark
#!pip install findspark

In [83]:
import findspark
import builtins
from pyspark.ml import Pipeline

findspark.init()
from cProfile import label

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, month, dayofmonth,abs,
    avg, sum, min, max, stddev, variance, percentile_approx,when
)
from pyspark.sql.functions import abs as spark_abs
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [84]:
# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("Rainfall_Prediction_Project") \
    .getOrCreate()


In [85]:
df = spark.read.csv("D:/SPARK/rainfall_fixed.csv", header=True, inferSchema=True)


In [86]:
# Đếm số dòng và cột
num_rows = df.count()
num_cols = len(df.columns)
print(f"Dataset có {num_rows} dòng và {num_cols} cột")

Dataset có 10000 dòng và 8 cột


In [87]:
# Xem 10 dòng đầu
df.show(10)

+-------------------+--------+-----------+-------------+------------+--------------+------------+-------------+
|               date|location|rainfall_mm|temperature_c|humidity_pct|wind_speed_kmh|pressure_hpa|rain_category|
+-------------------+--------+-----------+-------------+------------+--------------+------------+-------------+
|2020-01-01 00:00:00|  Danang|       7.43|         27.5|        50.0|          31.9|       997.9|     Moderate|
|2020-01-01 01:00:00|    HCMC|      12.56|         26.1|        80.7|          10.9|      1006.1|        Heavy|
|2020-01-01 02:00:00|     Hue|      10.79|         20.3|        60.3|          22.4|      1004.2|        Heavy|
|2020-01-01 03:00:00|     Hue|      11.22|         22.1|        86.4|           9.3|       997.5|        Heavy|
|2020-01-01 04:00:00|  Danang|       2.02|         24.8|        54.1|          25.9|      1007.4|     Moderate|
|2020-01-01 05:00:00|    HCMC|       2.86|         29.7|        57.1|          30.3|      1001.4|     Mo

In [88]:
# Kiểm tra cấu trúc dữ liệu
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- location: string (nullable = true)
 |-- rainfall_mm: double (nullable = true)
 |-- temperature_c: double (nullable = true)
 |-- humidity_pct: double (nullable = true)
 |-- wind_speed_kmh: double (nullable = true)
 |-- pressure_hpa: double (nullable = true)
 |-- rain_category: string (nullable = true)



In [89]:
# Khảo sát nhanh dữ liệu lượng mưa
df.describe("rainfall_mm").show()

+-------+-----------------+
|summary|      rainfall_mm|
+-------+-----------------+
|  count|            10000|
|   mean| 7.44524399999998|
| stddev|5.798614199369287|
|    min|              0.0|
|    max|            29.81|
+-------+-----------------+



In [90]:
# Thống kê mô tả chi tiết để đánh giá phân bố và quyết định chuẩn hóa
rain_stats = df.select(
    max('rainfall_mm').alias('max_rain'),
    min('rainfall_mm').alias('min_rain'),
    avg('rainfall_mm').alias('mean_rain'),
    stddev('rainfall_mm').alias('std_rain'),
    variance('rainfall_mm').alias('var_rain'),
    percentile_approx('rainfall_mm', 0.25).alias('Q1'),
    percentile_approx('rainfall_mm', 0.5).alias('Median'),
    percentile_approx('rainfall_mm', 0.75).alias('Q3')
)

rain_stats.show(truncate=False)


+--------+--------+----------------+-----------------+-----------------+----+------+-----+
|max_rain|min_rain|mean_rain       |std_rain         |var_rain         |Q1  |Median|Q3   |
+--------+--------+----------------+-----------------+-----------------+----+------+-----+
|29.81   |0.0     |7.44524399999998|5.798614199369287|33.62392663312712|2.72|6.24  |10.96|
+--------+--------+----------------+-----------------+-----------------+----+------+-----+



In [91]:
# Chuyển cột date về kiểu Date
df = df.withColumn("date", F.to_date("date", "yyyy-MM-dd"))

# Tách năm, tháng, ngày để phân tích thời gian
df = df.withColumn("year", F.year("date")) \
       .withColumn("month", F.month("date")) \
       .withColumn("day", F.dayofmonth("date"))

df.select("date", "year", "month", "day").show(5)


+----------+----+-----+---+
|      date|year|month|day|
+----------+----+-----+---+
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
|2020-01-01|2020|    1|  1|
+----------+----+-----+---+
only showing top 5 rows


In [92]:
# Thêm cột season với 4 mùa
df = df.withColumn(
    "season",
    when(col("month").isin(3, 4, 5), "spring")
    .when(col("month").isin(6, 7, 8), "summer")
    .when(col("month").isin(9, 10, 11), "autumn")
    .otherwise("winter")
)

In [93]:
#Tiền xử lý và ép kiểu an toàn an toàn
numeric_cols = [
    "rainfall_mm",
    "temperature_c",
    "humidity_pct",
    "wind_speed_kmh",
    "pressure_hpa"
]

for c in numeric_cols:
    df = df.withColumn(
        c,
        F.when(
            F.col(c).rlike(r'^-?\d+(\.\d+)?$'),
            F.col(c).cast(DoubleType())
        ).otherwise(None)
    )

In [94]:
print(" KIỂM TRA GIÁ TRỊ THIẾU:")
print("-" * 40)

total_rows = df.count()
for column in df.columns:
    # Đếm số dòng có giá trị null
    null_count = df.filter(col(column).isNull()).count()
    if null_count > 0:
        null_percentage = (null_count / total_rows) * 100
        print(f"{column}: {null_count} giá trị thiếu ({null_percentage:.2f}%)")


 KIỂM TRA GIÁ TRỊ THIẾU:
----------------------------------------


In [95]:
# Xóa các dòng có giá trị thiếu
df_clean = df.dropna()
print(f"Đã xóa dòng có missing values. Số dòng còn lại: {df_clean.count()}")

Đã xóa dòng có missing values. Số dòng còn lại: 10000


In [96]:
print("PHÂN TÍCH LƯỢNG MƯA THEO KHU VỰC:")

# Tạo bảng tạm để sử dụng SQL
df_clean.createOrReplaceTempView("rainfall")


PHÂN TÍCH LƯỢNG MƯA THEO KHU VỰC:


In [97]:
# Phân tích theo khu vực
print("PHÂN TÍCH LƯỢNG MƯA THEO KHU VỰC")
spark.sql("""
    SELECT
        location,
        COUNT(*) as so_mau,
        ROUND(AVG(rainfall_mm), 2) as avg_rain,
        ROUND(MIN(rainfall_mm), 2) as min_rain,
        ROUND(MAX(rainfall_mm), 2) as max_rain,
        ROUND(STDDEV(rainfall_mm), 2) as std_rain
    FROM rainfall
    GROUP BY location
    ORDER BY avg_rain DESC
""").show(truncate=False)


PHÂN TÍCH LƯỢNG MƯA THEO KHU VỰC
+--------+------+--------+--------+--------+--------+
|location|so_mau|avg_rain|min_rain|max_rain|std_rain|
+--------+------+--------+--------+--------+--------+
|Can Tho |1941  |7.58    |0.0     |29.16   |5.94    |
|HCMC    |1971  |7.52    |0.0     |29.14   |5.85    |
|Hue     |1974  |7.43    |0.0     |28.94   |5.65    |
|Hanoi   |2044  |7.36    |0.0     |29.81   |5.9     |
|Danang  |2070  |7.34    |0.0     |29.48   |5.65    |
+--------+------+--------+--------+--------+--------+



In [98]:
print("LƯỢNG MƯA TRUNG BÌNH & TỔNG THEO KHU VỰC")

spark.sql("""
    SELECT
        location,
        ROUND(AVG(rainfall_mm), 2) AS avg_rainfall,
        ROUND(SUM(rainfall_mm), 2) AS total_rainfall
    FROM rainfall
    GROUP BY location
    ORDER BY total_rainfall DESC
""").show(truncate=False)


LƯỢNG MƯA TRUNG BÌNH & TỔNG THEO KHU VỰC
+--------+------------+--------------+
|location|avg_rainfall|total_rainfall|
+--------+------------+--------------+
|Danang  |7.34        |15200.34      |
|Hanoi   |7.36        |15043.32      |
|HCMC    |7.52        |14827.13      |
|Can Tho |7.58        |14714.16      |
|Hue     |7.43        |14667.49      |
+--------+------------+--------------+



In [99]:
# Xếp hạng khu vực mưa nhiều nhất theo năm
print("XẾP HẠNG LƯỢNG MƯA THEO NĂM")
spark.sql("""
    SELECT
        year,
        location,
        ROUND(AVG(rainfall_mm),2) AS avg_rainfall,
        RANK() OVER (
            PARTITION BY year
            ORDER BY AVG(rainfall_mm) DESC
        ) AS rainfall_rank
    FROM rainfall
    GROUP BY year, location
    ORDER BY year, rainfall_rank
""").show()


XẾP HẠNG LƯỢNG MƯA THEO NĂM
+----+--------+------------+-------------+
|year|location|avg_rainfall|rainfall_rank|
+----+--------+------------+-------------+
|2020| Can Tho|        7.57|            1|
|2020|    HCMC|        7.53|            2|
|2020|     Hue|        7.41|            3|
|2020|   Hanoi|        7.36|            4|
|2020|  Danang|        7.35|            5|
|2021| Can Tho|        7.65|            1|
|2021|     Hue|        7.58|            2|
|2021|    HCMC|        7.48|            3|
|2021|   Hanoi|        7.34|            4|
|2021|  Danang|         7.3|            5|
+----+--------+------------+-------------+



In [100]:
# Phân tích theo mùa
print("\nPHÂN TÍCH LƯỢNG MƯA THEO MÙA:")
spark.sql("""
    SELECT
        season,
        COUNT(*) as count,
        ROUND(AVG(rainfall_mm), 2) as avg_rain
    FROM rainfall
    GROUP BY season
""").show()


PHÂN TÍCH LƯỢNG MƯA THEO MÙA:
+------+-----+--------+
|season|count|avg_rain|
+------+-----+--------+
|winter| 3400|    7.49|
|summer| 2208|    7.38|
|spring| 2208|    7.37|
|autumn| 2184|     7.5|
+------+-----+--------+



In [101]:
print("PHÂN LOẠI MỨC ĐỘ MƯA")

spark.sql("""
    SELECT
        rain_level,
        COUNT(*) AS total_days,
        ROUND(AVG(rainfall_mm), 2) AS avg_rainfall
    FROM (
        SELECT
            rainfall_mm,
            CASE
                WHEN rainfall_mm = 0 THEN 'No Rain'
                WHEN rainfall_mm < 10 THEN 'Light Rain'
                WHEN rainfall_mm < 30 THEN 'Moderate Rain'
                ELSE 'Heavy Rain'
            END AS rain_level
        FROM rainfall
    )
    GROUP BY rain_level
    ORDER BY avg_rainfall DESC
""").show(truncate=False)


PHÂN LOẠI MỨC ĐỘ MƯA
+-------------+----------+------------+
|rain_level   |total_days|avg_rainfall|
+-------------+----------+------------+
|Moderate Rain|2910      |14.99       |
|Light Rain   |7020      |4.39        |
|No Rain      |70        |0.0         |
+-------------+----------+------------+



In [102]:
print("SO SÁNH KHU VỰC VS TRUNG BÌNH TOÀN CỤC")

spark.sql("""
    WITH global_avg AS (
        SELECT AVG(rainfall_mm) AS global_avg
        FROM rainfall
    )
    SELECT
        r.location,
        ROUND(AVG(r.rainfall_mm), 2) AS location_avg,
        ROUND(g.global_avg, 2) AS global_avg,
        ROUND(AVG(r.rainfall_mm) - g.global_avg, 2) AS deviation
    FROM rainfall r
    CROSS JOIN global_avg g
    GROUP BY r.location, g.global_avg
    ORDER BY deviation DESC
""").show(truncate=False)


SO SÁNH KHU VỰC VS TRUNG BÌNH TOÀN CỤC
+--------+------------+----------+---------+
|location|location_avg|global_avg|deviation|
+--------+------------+----------+---------+
|Can Tho |7.58        |7.45      |0.14     |
|HCMC    |7.52        |7.45      |0.08     |
|Hue     |7.43        |7.45      |-0.01    |
|Hanoi   |7.36        |7.45      |-0.09    |
|Danang  |7.34        |7.45      |-0.1     |
+--------+------------+----------+---------+



In [103]:
print("PHÁT HIỆN NGÀY MƯA BẤT THƯỜNG")

spark.sql("""
    WITH stats AS (
        SELECT
            location,
            AVG(rainfall_mm) AS avg_rain,
            STDDEV(rainfall_mm) AS std_rain
        FROM rainfall
        GROUP BY location
    )
    SELECT
        r.date,
        r.location,
        r.rainfall_mm,
        ROUND(s.avg_rain, 2) AS avg_rain,
        ROUND(s.std_rain, 2) AS std_rain
    FROM rainfall r
    JOIN stats s
        ON r.location = s.location
    WHERE r.rainfall_mm > s.avg_rain + 2 * s.std_rain
    ORDER BY r.rainfall_mm DESC
""").show(truncate=False)


PHÁT HIỆN NGÀY MƯA BẤT THƯỜNG
+----------+--------+-----------+--------+--------+
|date      |location|rainfall_mm|avg_rain|std_rain|
+----------+--------+-----------+--------+--------+
|2021-02-07|Hanoi   |29.81      |7.36    |5.9     |
|2020-03-04|Danang  |29.48      |7.34    |5.65    |
|2020-05-02|Danang  |29.35      |7.34    |5.65    |
|2020-10-31|Can Tho |29.16      |7.58    |5.94    |
|2020-04-26|HCMC    |29.14      |7.52    |5.85    |
|2020-11-25|Hue     |28.94      |7.43    |5.65    |
|2020-06-18|HCMC    |28.54      |7.52    |5.85    |
|2020-05-04|Hanoi   |28.54      |7.36    |5.9     |
|2020-08-13|Hanoi   |28.26      |7.36    |5.9     |
|2020-10-31|Can Tho |28.14      |7.58    |5.94    |
|2020-06-18|Hue     |27.93      |7.43    |5.65    |
|2020-03-08|Danang  |27.54      |7.34    |5.65    |
|2020-09-25|HCMC    |27.51      |7.52    |5.85    |
|2021-01-22|Danang  |27.34      |7.34    |5.65    |
|2020-08-20|Can Tho |27.15      |7.58    |5.94    |
|2020-12-11|Danang  |27.1       |7

In [104]:
print("XU HƯỚNG LƯỢNG MƯA – TRUNG BÌNH TRƯỢT 7 NGÀY")

spark.sql("""
    SELECT
        date,
        location,
        rainfall_mm,
        ROUND(
            AVG(rainfall_mm) OVER (
                PARTITION BY location
                ORDER BY date
                ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
            ), 2
        ) AS rainfall_7day_avg
    FROM rainfall
    ORDER BY location, date
""").show(truncate=False)


XU HƯỚNG LƯỢNG MƯA – TRUNG BÌNH TRƯỢT 7 NGÀY
+----------+--------+-----------+-----------------+
|date      |location|rainfall_mm|rainfall_7day_avg|
+----------+--------+-----------+-----------------+
|2020-01-01|Can Tho |13.11      |13.11            |
|2020-01-01|Can Tho |0.51       |6.81             |
|2020-01-01|Can Tho |2.07       |5.23             |
|2020-01-02|Can Tho |11.38      |6.77             |
|2020-01-02|Can Tho |4.02       |6.22             |
|2020-01-02|Can Tho |2.85       |5.66             |
|2020-01-02|Can Tho |0.82       |4.97             |
|2020-01-02|Can Tho |0.51       |3.17             |
|2020-01-03|Can Tho |23.58      |6.46             |
|2020-01-03|Can Tho |6.49       |7.09             |
|2020-01-03|Can Tho |4.26       |6.08             |
|2020-01-04|Can Tho |15.19      |7.67             |
|2020-01-04|Can Tho |14.81      |9.38             |
|2020-01-04|Can Tho |2.6        |9.63             |
|2020-01-06|Can Tho |6.4        |10.48            |
|2020-01-06|Can Tho

TRANSFORM

In [105]:
# Chuẩn bị biến số
numeric_features = ["temperature_c", "humidity_pct",
                    "wind_speed_kmh", "pressure_hpa"]
label_column = "rainfall_mm"

print(f"\n1. Biến số (numeric): {numeric_features}")
print(f"2. Target: {label_column}")


1. Biến số (numeric): ['temperature_c', 'humidity_pct', 'wind_speed_kmh', 'pressure_hpa']
2. Target: rainfall_mm


In [106]:
# Mã hóa biến phân loại 'location'
indexer = StringIndexer(inputCol="location", outputCol="location_index")
encoder = OneHotEncoder(inputCol="location_index",
                       outputCol="location_encoded")

In [107]:
# Mã hóa biến 'season'
season_indexer = StringIndexer(inputCol="season", outputCol="season_index")
season_encoder = OneHotEncoder(inputCol="season_index",
                              outputCol="season_encoded")


In [108]:
    # Tạo danh sách feature cuối cùng
final_features = numeric_features + ["location_encoded", "season_encoded"]

print("CÁC FEATURE SẼ SỬ DỤNG:")
for i, feat in enumerate(final_features, 1):
    print(f"  {i}. {feat}")

CÁC FEATURE SẼ SỬ DỤNG:
  1. temperature_c
  2. humidity_pct
  3. wind_speed_kmh
  4. pressure_hpa
  5. location_encoded
  6. season_encoded


In [109]:
# Tạo Vector Assembler
assembler = VectorAssembler(
    inputCols=final_features,
    outputCol="features"
)

In [110]:
# Tạo pipeline HOÀN CHỈNH
feature_pipeline = Pipeline(stages=[
    indexer, encoder,
    season_indexer, season_encoder,
    assembler
])

In [111]:
#Fit và transform qua pipeline
df_transformed = feature_pipeline.fit(df_clean).transform(df_clean)

In [112]:
# Chọn chỉ columns cần thiết
df_final = df_transformed.select("features", label_column)

In [113]:
print("KẾT QUẢ TRANSFORM:")
print(f"Số mẫu: {df_final.count():,} mẫu")
print(f"Cấu trúc: {len(df_final.columns)} cột")
print(f"Cột: {df_final.columns}")

print("\n5 MẪU ĐẦU TIÊN (features đã encoded):")
print("-"*60)
df_final.show(5, truncate=False)


KẾT QUẢ TRANSFORM:
Số mẫu: 10,000 mẫu
Cấu trúc: 2 cột
Cột: ['features', 'rainfall_mm']

5 MẪU ĐẦU TIÊN (features đã encoded):
------------------------------------------------------------
+--------------------------------------------------+-----------+
|features                                          |rainfall_mm|
+--------------------------------------------------+-----------+
|(11,[0,1,2,3,4,8],[27.5,50.0,31.9,997.9,1.0,1.0]) |7.43       |
|(11,[0,1,2,3,7,8],[26.1,80.7,10.9,1006.1,1.0,1.0])|12.56      |
|(11,[0,1,2,3,6,8],[20.3,60.3,22.4,1004.2,1.0,1.0])|10.79      |
|(11,[0,1,2,3,6,8],[22.1,86.4,9.3,997.5,1.0,1.0])  |11.22      |
|(11,[0,1,2,3,4,8],[24.8,54.1,25.9,1007.4,1.0,1.0])|2.02       |
+--------------------------------------------------+-----------+
only showing top 5 rows


In [114]:
# Lấy một mẫu để xem cấu trúc
sample_row = df_final.limit(1).collect()[0]
features_vector = sample_row["features"]
rainfall_value = sample_row[label_column]

print(f"1. Rainfall (target gốc): {rainfall_value} mm")
print(f"2. Số lượng features: {len(features_vector)}")
print(f"3. Các features giữ nguyên scale gốc:")
print(f"   - Numeric features: {numeric_features}")
print(f"   - One-hot encoded: location (5 categories) + season (4 categories)")
print(f"4. Ví dụ vector features: {features_vector}")

1. Rainfall (target gốc): 7.43 mm
2. Số lượng features: 11
3. Các features giữ nguyên scale gốc:
   - Numeric features: ['temperature_c', 'humidity_pct', 'wind_speed_kmh', 'pressure_hpa']
   - One-hot encoded: location (5 categories) + season (4 categories)
4. Ví dụ vector features: (11,[0,1,2,3,4,8],[27.5,50.0,31.9,997.9,1.0,1.0])


In [115]:
def train_rainfall_model(
    df_final,
    feature_columns,
    label_column="rainfall_mm",
    test_size=0.2,
    seed=42,
    scale=True
):
    print("BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH")
    print(f"Test size: {test_size*100}%")

    # Kiểm tra dữ liệu
    print("\nKIỂM TRA DỮ LIỆU")
    if "features" not in df_final.columns:
        raise ValueError("Thiếu column 'features'")
    if label_column not in df_final.columns:
        raise ValueError(f"Thiếu column '{label_column}'")

    # Chia dữ liệu
    train_df, test_df = df_final.randomSplit([1 - test_size, test_size], seed=seed)
    print(f"  Train: {train_df.count()} mẫu")
    print(f"  Test : {test_df.count()} mẫu")

    # Khở tạo các stages
    stages = []
    if scale:
        scaler = StandardScaler(
            inputCol="features",
            outputCol="scaled_features",
            withMean=True,
            withStd=True
        )
        stages.append(scaler)
        features_col = "scaled_features"
    else:
        features_col = "features"

    # Khởi tạo thuật toán Linear Regression
    lr = LinearRegression(
        featuresCol=features_col,
        labelCol=label_column,
        solver="normal"
    )
    stages.append(lr)

    pipeline = Pipeline(stages=stages)

    # Huấn luyện mô hình
    print("\nHUẤN LUYỆN MÔ HÌNH...")
    pipeline_model = pipeline.fit(train_df)

    # lr là stage cuối cùng trong pipeline
    lr_model = pipeline_model.stages[-1]

    # Dự đoán và đánh giá
    predictions = pipeline_model.transform(test_df)

    print("\nĐÁNH GIÁ TRÊN TẬP TEST")
    evaluators = {
        "RMSE": RegressionEvaluator(labelCol=label_column, metricName="rmse"),
        "MAE": RegressionEvaluator(labelCol=label_column, metricName="mae"),
        "R2": RegressionEvaluator(labelCol=label_column, metricName="r2")
    }

    metrics = {}
    for name, evaluator in evaluators.items():
        value = evaluator.evaluate(predictions)
        metrics[name] = value
        print(f"  {name}: {value:.4f}")

    # Thống kế trên tập train
    print("\nTHỐNG KÊ TRÊN TẬP TRAIN")
    summary = lr_model.summary
    print(f"  R2 (train): {summary.r2:.4f}")
    print(f"  RMSE (train): {summary.rootMeanSquaredError:.4f}")

    importance = lr_model.coefficients.toArray().tolist()

    print("\nHOÀN THÀNH HUẤN LUYỆN")
    return {
        "pipeline_model": pipeline_model,
        "linear_model": lr_model,
        "predictions": predictions,
        "metrics": metrics,
        "feature_importance": importance
    }

In [116]:
results = train_rainfall_model(
    df_final=df_final,
    feature_columns=final_features,
    label_column="rainfall_mm",
    test_size=0.2,
    seed=42,
    scale=True   # ← có thể False để so sánh
)


BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH
Test size: 20.0%

KIỂM TRA DỮ LIỆU
  Train: 8079 mẫu
  Test : 1921 mẫu

HUẤN LUYỆN MÔ HÌNH...

ĐÁNH GIÁ TRÊN TẬP TEST
  RMSE: 4.6398
  MAE: 3.6295
  R2: 0.3355

THỐNG KÊ TRÊN TẬP TRAIN
  R2 (train): 0.3253
  RMSE (train): 4.7821

HOÀN THÀNH HUẤN LUYỆN


In [117]:
results = train_rainfall_model(
    df_final=df_final,
    feature_columns=final_features,
    label_column="rainfall_mm",
    test_size=0.2,
    seed=142,
    scale=True   # ← có thể False để so sánh
)


BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH
Test size: 20.0%

KIỂM TRA DỮ LIỆU
  Train: 7994 mẫu
  Test : 2006 mẫu

HUẤN LUYỆN MÔ HÌNH...

ĐÁNH GIÁ TRÊN TẬP TEST
  RMSE: 4.8431
  MAE: 3.7911
  R2: 0.3275

THỐNG KÊ TRÊN TẬP TRAIN
  R2 (train): 0.3274
  RMSE (train): 4.7325

HOÀN THÀNH HUẤN LUYỆN


In [118]:
spark.stop()